In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.make.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Make No'] = df_input['Make No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request = crawler_series['Make Url']

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath(f'''//b/a[@href="{crawler_series['Make Url']}"]/text()''')[0] == crawler_series['Make Text']:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_a = html.xpath('//a[@class="subcategory_link"]')

        # = = = = = = = = = = = = = = =

        if not list_a:
            raise

        # = = = = = = = = = = = = = = =

        list_url, list_text = [a.xpath('./@href')[0].strip() for a in list_a], [a.xpath('./span[@class="subcategory_name"]/text()')[0].strip() for a in list_a]

        # = = = = = = = = = = = = = = =

        list_year = [int(text.split(' ')[0].strip()) for text in list_text]

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend([{'Make No': crawler_series['Make No'],
                                       'Make': crawler_series['Make'],
                                       'Year': year,
                                       'Year Text': text,
                                       'Year Url': url} for year, text, url in zip(list_year, list_text, list_url)])

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Make No']}. {crawler_series['Make']}. {url_request}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Make No', 'Year'],
                                                             ascending=[True, False],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/2.year-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Make No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/year_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：53

[True] - [请求1次] - [剩余52条] - [2024/11/25 11:20:58] - 44. Saab. https://www.tpms.com/Saab-TPMS-s/1861.htm

[True] - [请求1次] - [剩余51条] - [2024/11/25 11:20:58] - 13. Fiat. https://www.tpms.com/Fiat-TPMS-s/5096.htm

[True] - [请求1次] - [剩余50条] - [2024/11/25 11:20:58] - 10. Chrysler. https://www.tpms.com/Chrysler-TPMS-s/1837.htm

[True] - [请求1次] - [剩余49条] - [2024/11/25 11:20:58] - 6. BMW. https://www.tpms.com/BMW-TPMS-s/1833.htm

[True] - [请求1次] - [剩余48条] - [2024/11/25 11:20:58] - 39. Pontiac. https://www.tpms.com/Pontiac-TPMS-s/1859.htm

[True] - [请求1次] - [剩余47条] - [2024/11/25 11:20:59] - 15. Genesis. https://www.tpms.com/Genesis-TPMS-s/10965.htm

[True] - [请求1次] - [剩余46条] - [2024/11/25 11:20:59] - 8. Cadillac. https://www.tpms.com/Cadillac-TPMS-s/1835.htm

[True] - [请求1次] - [剩余45条] - [2024/11/25 11:20:59] - 4. Audi. https://www.tpms.com/Audi-TPMS-s/1831.htm

[True] - [请求1次] - [剩余44条] - [2024/11/25 11:20:59] - 16. GMC. https://www.tpms.com/GMC-TPMS-s/1841.htm

[True] - [请求1次] - [剩余43条]